<a href="https://colab.research.google.com/github/lrsilsbee/Machine-Learning/blob/main/Lauren_Silsbee_feature_engineering_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
import warnings
import seaborn as sns
sns.set
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

warnings.filterwarnings('ignore')
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

us_education = pd.read_sql_query('select * from useducation',con=engine)
engine.dispose()

Question 1: Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [ ]:
#Handle missing values first:

fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = us_education["STATE"].unique()

for state in states:
    us_education.loc[us_education["STATE"] == state, fill_list] = us_education.loc[us_education["STATE"] == state, fill_list].interpolate()

# we drop the null values after interpolation
us_education.dropna(inplace=True)

In [ ]:
#Create a new variable with weighted averages and add to data frame

us_education["total_score"] = (us_education["GRADES_4_G"]*((us_education["AVG_MATH_4_SCORE"] + us_education["AVG_READING_4_SCORE"])*0.5) + us_education["GRADES_8_G"]
                                 * ((us_education["AVG_MATH_8_SCORE"] + us_education["AVG_READING_8_SCORE"])*0.5))/(us_education["GRADES_4_G"] + us_education["GRADES_8_G"])

Question 2: What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [ ]:
us_education.head()
#TOTAL_EXPENDITURE
#INSTRUCTION_EXPENDITURE
#SUPPORT_SERVICES_EXPENDITURE
#OTHER_EXPENDITURE
#CAPITAL_OUTLAY_EXPENDITURE


us_education[["total_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

print('Of expenditure types, instruction expenditure is most correlated with total score, though this is still quite a low correlation number.')

Of expenditure types, instruction expenditure is most correlated with total score, though this is still quite a low correlation number.


Question 3: Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [ ]:
#Use PCA with just the expenditure variables. Then look at the first component

#Matrix X with the four variables: 
X = us_education[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

#standarize the variables in matrix X:
X = StandardScaler().fit_transform(X)

#Conduct PCA, look at just the first component: 
sklearn_pca = PCA(n_components=1)
us_education["pca_1"] = sklearn_pca.fit_transform(X)



print(sklearn_pca.explained_variance_ratio_)
print('The variance explained by this first component is 94 %')

[0.9430175]
The variance explained by this first component is 94 %


Question 4: What is the correlation between the overall score variable and the 1st principal component?

In [ ]:
us_education[['total_score', "pca_1"]].corr()

,total_score,pca_1
total_score,1.000000,0.044596
pca_1,0.044596,1.000000


Question 5: If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In [ ]:
us_education[['total_score', "pca_1", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()
print('Instruction expenditure is more correlated with total score than is the first PCA component, so we should use the instruction expenditure variable.')

Instruction expenditure is more correlated with total score than is the first PCA component, so we should use the instruction expenditure variable.
